In [81]:
#-*- coding: utf-8 -*-

from time import sleep
import requests
import json
import re
from pytz import timezone
from datetime import datetime
from urllib.parse import urlencode
from bs4 import BeautifulSoup

#import urllib3
#urllib3.disable_warnings()

In [82]:
def get_tvN(day_list):
    #tvN
    #https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22404932610284963639_1588311372472&key=SingleChannelDailySchedule&where=m&pkid=66&u1=814825&u2=20200501

    total = 0
    program_list = set()
        
    for craw_day in day_list:
        base_url = 'https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22404932610284963639_1588311372472&key=SingleChannelDailySchedule&where=m&pkid=66&u1=814825&u2=' + str(craw_day)
        
        arr = base_url.split('?')
        arr2 = arr[1].split('&')
        skip_word = (arr2[0].split('='))[1] + '( '
        #print (base_url)
        
        req = requests.get(base_url, verify=False)
        total_data1 = req.text
        total_data2 = total_data1.replace(skip_word, '')
        total_data3 = total_data2.replace(');', '')

        var = json.loads(total_data3)
        naver_list = var['dataHtml']

        for link in naver_list:
            soup = BeautifulSoup(link, "html.parser")
            total_data = soup.find_all(attrs={'class': 'lk'})
            total = total + len(total_data)

            for link in total_data:
                program_list.add(link.text)

            #print ('\n#################################\n')
            #print (program_list)
            #print ('\n===> program_list : ', len(program_list))

    return (program_list)

In [93]:
def get_general(day_list):
    channel_list = [
        #지상파
        'https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22406981672323418231_1588307496613&key=MultiChannelDailySchedule&where=m&pkid=66&u1=100&u2=&u3=1+2+3&u5=0&u4='    
        #종편
        ,
        'https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22408213267523362562_1588310643412&key=MultiChannelDailySchedule&where=m&pkid=66&u1=500&u2=&u3=&u5=0&u4='
        #케이블
        ,
        'https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22406981672323418231_1588307496613&key=MultiChannelDailySchedule&where=m&pkid=66&u1=200&u2=&u3=12&u5=0&u4='
        #스카이프
        #,
        #'https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22406981672323418231_1588307496613&key=MultiChannelDailySchedule&where=m&pkid=66&u1=300&u2=&u3=25&u5=0&u4='
        #라디오
        #,
        #'https://m.search.naver.com/p/csearch/content/nqapirender.nhn?callback=jQuery22406981672323418231_1588307496613&key=MultiChannelDailySchedule&where=m&pkid=66&u1=400&u2=&u3=&u5=0&u4='
    ]

    total = 0
    program_list = set()

    for base_url in channel_list:
        
        for craw_day in day_list:
            base_url = base_url + str(craw_day)

            arr = base_url.split('?')
            arr2 = arr[1].split('&')
            skip_word = (arr2[0].split('='))[1] + '( '
            #print (skip_word)

            req = requests.get(base_url, verify=False)
            total_data1 = req.text
            total_data2 = total_data1.replace(skip_word, '')
            total_data3 = total_data2.replace(');', '')

            var = json.loads(total_data3)
            naver_list = var['dataHtml']

            for link in naver_list:
                soup = BeautifulSoup(link, "html.parser")
                total_data = soup.find_all(attrs={'class': 'pr_name'})
                total = total + len(total_data)
                for p in total_data:
                    #print (p.text)
                    program_list.add(p.text)

            #print ('\n#################################\n')
            #print (program_list)
            #print ('\n===> program_list : ', len(program_list))

    return (program_list)


In [94]:
def get_pages():
    pages = set() # 애초에 set은 중복을 허용하지 않음 

    #지상파
    base_uri = 'https://m.search.naver.com/search.naver'
    base_url = 'https://m.search.naver.com/search.naver?where=m&query=%EB%B0%A9%EC%86%A1%ED%8E%B8%EC%84%B1%ED%91%9C&sm=mtb_she&qdt=0'
    req = requests.get(base_url, verify=False)
    soup = BeautifulSoup(req.text, "html.parser")
    total_data = soup.find(attrs={'class': 'channel_list'})
    
    craw_baseurl_list = set()
    for link in total_data.findAll("a"):
        if 'href' in link.attrs: 
            url = base_uri + link.attrs['href']
            print(url)
            pages.add(url)

    return (pages)

In [95]:
def craw_data(base_url):
    req = requests.get(base_url, verify=False)
    soup = BeautifulSoup(req.text, "html.parser")
    total_data = soup.find(attrs={'class': 'sc cs_tvtime _cs_tvtime'})
    total_data2 = total_data.find_all(attrs={'class': 'lk'})

    craw_index = 0
    craw_data = set()
    
    for d in total_data2:
        craw_index = craw_index + 1
        craw_data.add(d.text)
        print(craw_index, d.text)
        
    return (craw_data)

In [96]:
def clean_text(text):

    #(참고 사이트) https://data-newbie.tistory.com/210
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' 
    pattern = r'\([^)]*\)'
    clean_text = re.sub(pattern=pattern, repl='', string=text)
    #print("괄호와 괄호안 글자 제거 :  " , text , "\n")

    return (clean_text)

In [98]:
if __name__ == "__main__":
    fmt = "%Y-%m-%d %H:%M:%S %Z%z"
    #while (True):
    KST = datetime.now(timezone('Asia/Seoul'))
    
    program_list = set()
    
    day_list = [20200426, 20200427, 20200428, 20200429, 20200430, 20200501, 20200502, 20200503, 20200504, 20200505, 20200506]
    
    p_list = get_tvN(day_list)
    for p in p_list:
        program_list.add(clean_text(p))
    p_list = get_general(day_list)
    for p in p_list:
        program_list.add(clean_text(p))
    
    result_list = list(program_list)
    print ('len : ', len(result_list))
    for p in sorted(result_list):
        print (p)

/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

len :  502
100분 토론
12 MBC 뉴스
1박 2일
1박 2일 시즌4
20주년 기획 인간극장
2TV 생생정보
2시 뉴스 외전
365 : 운명을 거스르는 1년
48+ 좋은 친구들
5 MBC 뉴스
60분 드라마 2016
6시 내고향
930 MBC 뉴스
A특공대 2
BBB삼총사의 모험
CSI:라스베가스 10
EBS 뉴스
EBS 다큐프라임
EBS 마스터
EBS 명강
EBS 온라인개학 초등 국어 1-1 가
EBS 온라인개학 초등 국어 2-1 가
EBS 온라인개학 초등 수학 1-1
EBS 온라인개학 초등 수학 2-1
EBS 정오뉴스
EBS 초대석
EBS 클래식
EBSe 생활영어
JTBC NEWS 아침 &
JTBC 뉴스룸
JTBC 스페셜 다큐멘터리
JTBC 이 시각 뉴스룸
KBS 경제타임
KBS 네트워크 특선 고고씽
KBS 네트워크 특선 보물섬
KBS 네트워크 특선 콘서트 문화창고
KBS 네트워크 특선 호모사이언스
KBS 아침 뉴스타임
KBS 재난방송센터
KBS 중계석 스페셜
KBS 착한 소비 프로젝트 우리 함께
KBS걸작 다큐멘터리
MBC PICK X 놀면 뭐하니? 스페셜
MBC PICK X 오! 나의 파트,너
MBC 뉴스
MBC 뉴스데스크
MBC 뉴스투데이 1부
MBC 뉴스투데이 2부
MBC네트워크특선 푸른 백세 어영차 바다야
MBN 뉴스와이드
MBN 스페셜 배틀봇 4
MBN 종합뉴스
MBN 프레스룸
NCIS 시즌13
NCIS:LA 7
NCIS:LA 시즌8
NCIS:뉴올리언스 5
PD수첩
SBS 12뉴스
SBS 8 뉴스
SBS 뉴스
SBS 스페셜
SBS 오뉴스
SBS 인기가요
Sesame Street
TV CHOSUN 뉴스
TV CHOSUN 뉴스7
TV CHOSUN 뉴스9
TV CHOSUN 뉴스현장
TV 동물농장
TV 동물농장 1부
TV 동물농장 2부
TV 유치원
TV 정보쇼 알짜왕
TV 주치의 닥터 지.바.고
TV쇼 진품명품
TV예술무대
UHD 숨터
UHD 특선 다큐 하늘에서 본 세계
VJ 생생정보
[BTS월드투어특집2탄]브링 더 소울: 다큐시리즈1
[tv

/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
